In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sympy import latex
from IPython.display import display, Math
plt.rcParams['figure.figsize'] = [20, 10]

In [2]:
# retrived from https://www.arduino.cc/reference/en/language/functions/math/map/
def map_func(x, in_min, in_max, out_min, out_max):
    return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

In [16]:
def adc2angle(df, shoulder_min=203, soulder_max=599, elbow_min=865, elbow_max=261, 
              shoulder_min_angle =-90, shoudler_max_angle=0, 
              elbow_min_angle=0, elbow_max_angle=145):
    col = df.columns # extract column names from dataframe
    
    # convertes first column (shoulder) to angles between -90 and 0
    df[col[0]] = df.apply(lambda x: map_func(df[col[0]], shoulder_min, soulder_max, shoulder_min_angle, shoudler_max_angle)) 
    
    # convertes second column (elbow) to angles between 0 and 160
    df[col[1]] = df.apply(lambda x: map_func(df[col[1]], elbow_min, elbow_max, elbow_min_angle, elbow_max_angle))
    return df

# Mapping Calibration

In [19]:
df = pd.read_csv('./datasets/pot_cal_dataset_clean_1.csv') # dataset generated during calibration run
df # prints pot ADC values

,Shoulder,Elbow
0,203.0,490.0
1,203.0,490.0
2,198.0,489.0
3,202.0,865.0
4,599.0,491.0
5,598.0,490.0
6,598.0,489.0


- shoulder
    - min
        - angle:-90
        - ADC: 203
        
    - max
        - angle: 0
        - ADC: 599

- elbow
    - min
        - angle: 0
        - ADC: 865
    - max
        - angle: 90
            - extrapolated angle: 145
        - ADC: 490
            - extrapolated ADC: 261

In [20]:
m = (865 - 490)/(0 - 90)

y_145 = m*145 + 865
y_145

260.83333333333326

In [21]:
df['Shoulder'] = df.apply(lambda x: map_func(df['Shoulder'], 203, 599, -90, 0))
df['Elbow'] = df.apply(lambda x: map_func(df['Elbow'], 865, 261, 0, 145))
df # prints pot converted angle values

,Shoulder,Elbow
0,-90.000000,90.024834
1,-90.000000,90.024834
2,-91.136364,90.264901
3,-90.227273,0.000000
4,0.000000,89.784768
5,-0.227273,90.024834
6,-0.227273,90.264901


# Converting ADC Values of Traced Image to Angles

In [22]:
df = pd.read_csv('hyster_dataset_clean_1.csv') # second_trial_3.csv is the results of the traced image
df = adc2angle(df)
df.to_csv('angles.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'hyster_dataset_clean_1.csv'